In [2]:
# OPTIONAL: Load the "autoreload" extension so that code can change
%load_ext autoreload

# OPTIONAL: always reload modules so that as you change code in src, it gets loaded
%autoreload 2

In [3]:
import os

os.chdir('/Users/madsbirch/Documents/4_semester/mlops/mlops-sentiment-analysis')
print("Current working directory: {0}".format(os.getcwd()))

import gzip, json
import random
import numpy as np
import pandas as pd
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

from transformers import BertModel, BertTokenizer, DistilBertModel, DistilBertTokenizer



from src.data.AmazonReviewData import AmazonReviewsDataset
from src.data.make_dataset_temp import get_pandas_DF, sentiment_map, preprocess_data, get_dataloaders

torch.manual_seed(0)
np.random.seed(0)
random.seed(0)

# Set device (mps is specific to mac with M1 processor)
device = "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using device: {device}")

# path to raw data
raw_data_path = "data/raw/"


Current working directory: /Users/madsbirch/Documents/4_semester/mlops/mlops-sentiment-analysis
Using device: mps


### 1 Data exporation

The data can be downloaded from: http://jmcauley.ucsd.edu/data/amazon/links.html

In [4]:
df = get_pandas_DF(raw_data_path+'reviews_Automotive_5.json.gz')
# subset columns and rename to more intuitive names 
df = df[['overall', 'reviewText']]
df = df.rename(columns={'overall': 'sentiment', 'reviewText': 'review'})
  
# do sentiment mapping
df.sentiment = df.sentiment.apply(sentiment_map)
df.head()

,sentiment,review
0,2,I needed a set of jumper cables for my new car...
1,2,"These long cables work fine for my truck, but ..."
2,2,Can't comment much on these since they have no...
3,2,I absolutley love Amazon!!! For the price of ...
4,2,I purchased the 12' feet long cable set and th...


Classes are highly imbalanced!

In [5]:
df.sentiment.value_counts()

2    17895
1     1430
0     1148
Name: sentiment, dtype: int64

### Train model

In [6]:
train_loader, test_loader = get_dataloaders(batch_size = 12, num_workers = 0)

In [7]:
model = BertModel.from_pretrained('bert-base-cased')
#model = DistilBertModel.from_pretrained('distilbert-base-uncased')

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [8]:
# testing the model

for batch in train_loader:
    review = batch['review']
    input_ids = batch['input_ids']
    attention_mask = batch['attention_mask']
    labels = batch['labels']
    
    out = model(input_ids, attention_mask)
    pooled_out = out['pooler_output']
    break

In [9]:
pooled_out

tensor([[-0.7009,  0.4387,  0.9998,  ...,  0.9999, -0.7846,  0.9871],
        [-0.7625,  0.4366,  0.9999,  ...,  1.0000, -0.7165,  0.9935],
        [-0.7900,  0.4732,  0.9999,  ...,  1.0000, -0.7976,  0.9955],
        ...,
        [-0.7630,  0.4506,  0.9999,  ...,  1.0000, -0.7497,  0.9906],
        [-0.6988,  0.2968,  0.9996,  ...,  0.9999, -0.9025,  0.9821],
        [-0.6581,  0.3612,  0.9997,  ...,  0.9999, -0.5910,  0.9741]],
       grad_fn=<TanhBackward0>)

##### MODEL INFO

A description of the BERT model can be found at: https://huggingface.co/bert-base-uncased

We use the uncased model, which does not differ between Mads and mads (capital letters are ignored.)

It would be nice exploring how to download smaller pretrained models, as those would be trainable locally. A wide range of smaller models are available at: https://github.com/google-research/bert/



In [10]:
class BertSentiment(nn.Module):
  def __init__(self, n_classes: int, dropout: float, bert_out_dim = 768):
    super(BertSentiment, self).__init__()
    self.bert = BertModel.from_pretrained("bert-base-uncased")
    self.dropout = nn.Dropout(dropout)
    self.output = nn.Linear(bert_out_dim, n_classes)

  def forward(self, input_ids, attention_mask):
    output = self.bert(input_ids, attention_mask)
    pooled_output = output['pooler_output']
    x = self.dropout(pooled_output)
    out = self.output(x)
    
    return out

### NOT USED AND NOT WORKING. IF WE WANT TO USE THE OUTPUT OF THE MODEL NEEDS TO BE POOLED SOMEHOW ###
class DistilBertSentiment(nn.Module):
  def __init__(self, n_classes: int, dropout: float, bert_out_dim = 3072):
    super(DistilBertSentiment, self).__init__()
    self.bert = DistilBertModel.from_pretrained("distilbert-base-uncased")
    self.dropout = nn.Dropout(dropout)
    self.output = nn.Linear(bert_out_dim, n_classes)

  def forward(self, input_ids, attention_mask):
    output = self.bert(input_ids, attention_mask)
    pooled_output = output['pooler_output']
    x = self.dropout(pooled_output)
    out = self.output(x)
    
    return out


In [11]:
model = BertSentiment(n_classes=3, dropout=0.2).to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.AdamW(model.parameters(), lr=1e-4)
epochs = 5

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [17]:
# BASIC TRAINNIG LOOP

for epoch in range(epochs):
  print(f'[EPOCH]: {epoch:3d}')
  
  model.train()
  for batch in tqdm(train_loader):
    
    # move data to device
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    labels = batch['labels'].to(device)
    
    # get model outout and calc loss
    output = model(input_ids, attention_mask)
    loss = criterion(output, labels)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

[EPOCH]:   0


 26%|██▌       | 309/1195 [07:14<27:12,  1.84s/it]